In [1]:
import pandas as pd

from FlightRadar24.api import FlightRadar24API
fr_api = FlightRadar24API()

In [2]:
df_airlines = pd.DataFrame(fr_api.get_airlines())[["Name","ICAO"]]
df_airlines.to_parquet("Airlines.parquet")

In [3]:
df_airlines = pd.read_parquet("Airlines.parquet")
df_airlines

,Name,ICAO
0,21 Air,CSB
1,25only Aviation,ONY
2,2Excel Aviation,BRO
3,40-Mile Air,MLA
4,748 Air Services,IHO
...,...,...
1789,Zil Air,SYZ
1790,Zimex Aviation,IMX
1791,Zimex Aviation Austria,AZD
1792,ZIPAIR,TZP


In [56]:
df_airports=pd.read_parquet("Airports.parquet")
df_airports

,name,country,Continent,iata
0,\tMasai Mara Keekorok Airport,Kenya,africa,KEU
1,A Coruna Airport,Spain,europe,LCG
2,Aachen Merzbruck Airport,Germany,europe,AAH
3,Aalborg Airport,Denmark,europe,AAL
4,Aarhus Airport,Denmark,europe,AAR
...,...,...,...,...
4936,Zunyi Xinzhou Airport,China,asia,ZYI
4937,Zurich Airport,Switzerland,europe,ZRH
4938,Zwara Airport,Libya,africa,WAX
4939,Zweibrucken Airport,Germany,europe,ZQW


In [5]:
def active_airplane(a_icao):
    flight_list=[]
    flights = fr_api.get_flights(airline = a_icao)
    for flight in flights :
        try :
            dest_iata = flight.destination_airport_iata
        except :
            dest_iata = None
        try :
            org_iata = flight.origin_airport_iata
        except :
            org_iata = None
        try :
            dest_icao = flight.destination_airport_icao
        except :
            dest_icao = None
        try :
            org_icao = flight.origin_airport_icao
        except :
            org_icao = None
        try :
            model = flight.aircraft_model
        except :
            model = None 
        try :
            registration = flight.registration
        except :
            registration = None
        try :
            speed = flight.ground_speed
        except :
            speed = None

        f = {
            "id" : flight.id,
            "dest_icao" : dest_icao,
            "org_icao" : org_icao,
            "dest_iata" : dest_iata,
            "org_iata" : org_iata,
            "model" : model,
            "registration" : registration,
            "speed" : speed,
            "company" : a_icao
            }
        flight_list.append(f)
    return flight_list

In [6]:
active_flights = []
for airline in df_airlines["ICAO"]:
    active_flights += active_airplane(airline)

In [8]:
df_flights = pd.DataFrame(active_flights)
df_flights

,id,dest_icao,org_icao,dest_iata,org_iata,model,registration,speed,company
0,2da0a965,None,None,SBD,ABE,None,N750AX,0,ABX
1,2da0e28b,None,None,LAX,CVG,None,N364CM,378,ABX
2,2da0f3a5,None,None,ABE,SMF,None,N744AX,493,ABX
3,2da16a92,None,None,PHX,CVG,None,N797AX,392,ABX
4,2da1836a,None,None,IAH,CVG,None,N312AA,403,ABX
...,...,...,...,...,...,...,...,...,...
9292,2da18f37,None,None,ARN,GOT,None,OE-HZA,168,IMX
9293,2da18fc7,None,None,ARN,UME,None,HB-ALR,7,IMX
9294,2da0de56,None,None,NRT,BKK,None,JA822J,499,TZP
9295,2da128a4,None,None,NRT,LAX,None,JA826J,515,TZP


In [9]:
df_flights.to_parquet("Flights.parquet")

# Q2

In [59]:
df= pd.read_parquet("Flights.parquet",columns=["dest_iata","org_iata","company"])
airports = pd.read_parquet("Airports.parquet", columns=["iata","Continent"])

def same_continent(x):
    dest = x["dest_iata"]
    org = x["org_iata"]
    try :
        if airports[airports["iata"] == org]["Continent"].iloc[0] == airports[airports["iata"] == dest]["Continent"].iloc[0]:
            return airports[airports["iata"] == org]["Continent"].iloc[0]
        else :
            return None
    except IndexError as e:
        return None

S = df.apply(same_continent, axis=1)
S.sort_values()

4156    africa
6835    africa
4148    africa
2206    africa
6192    africa
         ...  
9285      None
9286      None
9294      None
9295      None
9296      None
Length: 9297, dtype: object

In [60]:
df["Continent"] = df.apply(same_continent, axis=1)

In [61]:
df

,dest_iata,org_iata,company,Continent
0,SBD,ABE,ABX,northamerica
1,LAX,CVG,ABX,northamerica
2,ABE,SMF,ABX,northamerica
3,PHX,CVG,ABX,northamerica
4,IAH,CVG,ABX,northamerica
...,...,...,...,...
9292,ARN,GOT,IMX,europe
9293,ARN,UME,IMX,europe
9294,NRT,BKK,TZP,None
9295,NRT,LAX,TZP,None


In [94]:
for continent in fr_api.get_zones().keys():
    count = df[df["Continent"] == continent]["company"].value_counts()
    if not count.empty:
        print(continent.upper(), ":\n", df_airlines[df_airlines["ICAO"] == count.index[0]].iloc[0]["Name"] , ":", count.max())

EUROPE :
 Ryanair : 272
NORTHAMERICA :
 American Airlines : 771
SOUTHAMERICA :
 LATAM Airlines : 105
OCEANIA :
 Air New Zealand : 49
ASIA :
 IndiGo : 44
AFRICA :
 Binter Canarias : 5
NORTHATLANTIC :
 Air Greenland : 2


In [66]:
fr_api.get_zones().keys()

dict_keys(['europe', 'northamerica', 'southamerica', 'oceania', 'asia', 'africa', 'atlantic', 'maldives', 'northatlantic'])

# Q3

In [96]:
def distance(x1,x2,y1,y2):
    return ((x1-x2)**2+(y1-y2)**2)**0.5

def airport_dist(x):
    org = x["org_iata"]
    dest = x["dest_iata"]
    

In [97]:
df3 = pd.read_parquet("Flights.parquet",columns=["dest_iata","org_iata"])
df3

,dest_iata,org_iata
0,SBD,ABE
1,LAX,CVG
2,ABE,SMF
3,PHX,CVG
4,IAH,CVG
...,...,...
9292,ARN,GOT
9293,ARN,UME
9294,NRT,BKK
9295,NRT,LAX
